In [77]:
# CREATE SYNTHETIC DATA

import pandas as pd
import numpy as np

# Create a template DataFrame with the same structure as your original data
# Replace this with the actual column names and types from your original data

your_path = 'C:/Users/OskarHansson/trioptima/'
file_path = your_path + '3.Cash_Risk/Modified_IRS_All_Train_VF_Cash_Risk.csv'

cols_to_modify = [
"total_delta",
"MtM_leg2",
"MtM_leg1",
"leg1NotionalAmountUSD",
"leg1FixedRate",
"leg2NotionalAmount",
"leg1NotionalAmount"]

data = pd.read_csv(file_path)

# Number of duplicates
num_duplicates = 1

# List to store duplicated rows
synthetic_data = []

# Loop through each row and create duplicates with noise
i = 0
for index, row in data.iterrows():
    for _ in range(num_duplicates):
        new_row = row.copy()
        i += 1

        for col_to_modify in cols_to_modify:
            # Create a new row by adding noise to the data    
            new_row[col_to_modify] += np.random.normal(0, abs(new_row[col_to_modify]) * 0.05)
            
        synthetic_data.append(new_row)


# Concatenate the original data with the synthetic data
synthetic_data = pd.concat([data, pd.DataFrame(synthetic_data)], ignore_index=True)

# Save the synthetic data to a new CSV file
synthetic_data.to_csv(your_path + "3.Cash_Risk/synthetic_data.csv", index=False)


C:\Users\OskarHansson\AppData\Local\Temp\ipykernel_15720\660110282.py:21: DtypeWarning: Columns (1,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import joblib
import os

#r'C:/Users/gusta/Documents/KTH/TriOptima/trioptima/trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
your_path = 'C:/Users/OskarHansson/trioptima/'

# Load the data
data = pd.read_csv(your_path + "3.Cash_Risk\Modified_IRS_All_Train_VF_Cash_Risk.csv")

# Step 1: Handle Missing Data
data_filled = data.fillna(0)

# Step 2: Extract Information from DateTime Columns
datetime_columns = ['effectiveDate', 'executionDateTime', 'expirationDate'] # ['effectiveDate', 'expirationDate', 'eventDateTime',] # 

def extract_date_features(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce')
    df[column + '_year'] = df[column].dt.year
    df[column + '_month'] = df[column].dt.month
    df[column + '_day'] = df[column].dt.day
    df[column + '_hour'] = df[column].dt.hour
    df[column + '_minute'] = df[column].dt.minute
    df[column + '_second'] = df[column].dt.second
    df[column + '_weekday'] = df[column].dt.weekday
    # Drop the original datetime column
    df = df.drop(column, axis=1)
    return df

for col in datetime_columns:
    data_filled = extract_date_features(data_filled, col)

original_dtypes = data.dtypes.to_dict()

# Convert boolean columns to binary (1/0) before one-hot encoding other categorical columns
data_filled = data_filled*1

def compare_item(row,leg1,leg2):
    return 1 if row[leg1] == row[leg2] else 0

cur = data_filled.apply(lambda row: compare_item(row, 'leg1NotionalCurrency','leg2NotionalCurrency'), axis=1)
nom = data_filled.apply(lambda row: compare_item(row, 'leg1NotionalAmount','leg2NotionalAmount'), axis=1)

# Step 3: Encode Categorical Variables
categorical_columns = data_filled.select_dtypes(include=['object', 'bool']).columns.tolist()
categorical_columns = [col for col in categorical_columns if col not in datetime_columns]
data_encoded = pd.get_dummies(data_filled, columns=categorical_columns, drop_first=True)

# Step 3.5: Identify and Drop Single-Value Columns
single_value_columns = data_encoded.columns[data_encoded.nunique() == 1].tolist()

# Optionally print these columns and their unique values
print("Columns with a single unique value: ", single_value_columns)

# Drop the single-value columns
data_encoded = data_encoded.drop(columns=single_value_columns)
data_encoded['CurrencyIV'] = cur
data_encoded['NominalIV'] = nom
#data_encoded = data_encoded.fillna(0)
# Define numerical_columns here, after all column dropping and adding has occurred



print(data_encoded.columns)
# Create interaction variable


numerical_columns = data_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Check for NaN or infinite values in numerical columns before scaling
nan_inf_columns = data_encoded[numerical_columns].columns[data_encoded[numerical_columns].isna().any() | np.isinf(data_encoded[numerical_columns]).any()].tolist()
print("Columns with NaN or infinite values before scaling: ", nan_inf_columns)

# Step 4: Scale/Normalize Data
numerical_columns = data_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
scaler = RobustScaler()
data_encoded[numerical_columns] = scaler.fit_transform(data_encoded[numerical_columns])

# Save the scaler for future use
#scaler_filename = 'robust_scaler.pkl'
#joblib.dump(scaler, scaler_filename)

# Preprocessed data is now stored in `data_encoded`, and the scaler is saved as `robust_scaler.pkl`
# Save the processed data to a CSV file in this environment
data_encoded.to_csv(your_path + '4.Scaled/synthetic_data.csv', index=False)

train_encoded_columns = data_encoded.columns


C:\Users\OskarHansson\AppData\Local\Temp\ipykernel_15720\645665953.py:12: DtypeWarning: Columns (1,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(your_path + "3.Cash_Risk\Modified_IRS_All_Train_VF_Cash_Risk.csv")


AttributeError: Can only use .dt accessor with datetimelike values